In [ ]:
#installation

In [1]:
!pip install mediapipe opencv-python

  Obtaining dependency information for mediapipe from https://files.pythonhosted.org/packages/29/05/acb01eb7c5920f50bd9bb748771dd5c8aef9c2555332c8d57da0e19500b6/mediapipe-0.9.1.0-cp311-cp311-macosx_10_15_x86_64.whl.metadata
  Using cached opencv_python-4.10.0.84-cp311-cp311-macosx_10_16_x86_64.whl
  Obtaining dependency information for absl-py from https://files.pythonhosted.org/packages/a2/ad/e0d3c824784ff121c03cc031f944bc7e139a8f1870ffd2845cc2dd76f6c4/absl_py-2.1.0-py3-none-any.whl.metadata
  Obtaining dependency information for attrs>=19.1.0 from https://files.pythonhosted.org/packages/e0/44/827b2a91a5816512fcaf3cc4ebc465ccd5d598c45cefa6703fcf4a79018f/attrs-23.2.0-py3-none-any.whl.metadata
  Obtaining dependency information for flatbuffers>=2.0 from https://files.pythonhosted.org/packages/41/f0/7e988a019bc54b2dbd0ad4182ef2d53488bb02e58694cd79d61369e85900/flatbuffers-24.3.25-py2.py3-none-any.whl.metadata
  Obtaining dependency information for matplotlib from https://files.pythonhoste

In [4]:
#Ignoriung Warnings
import sys

if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

In [5]:
#importing libs
import cv2
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

<img src="https://i.imgur.com/3j8BPdc.png" style="height:300px" >

In [6]:
# Calculating Angles:

def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle 

# Calculating Angle with Verticle

def calculate_angle_with_vertical(a, b):
    a = np.array(a)  # First point
    b = np.array(b)  # Mid point (vertex)

    # Calculate the vector from b to a
    vector = a - b

    # Angle of the vector with respect to the vertical axis (y-axis)
    # Vertical axis is along the y-axis, so we use (0, 1) as the vertical reference vector
    vertical_reference = np.array([0, 1])
    
    # Calculate the dot product and magnitudes of the vectors
    dot_product = np.dot(vector, vertical_reference)
    magnitude_vector = np.linalg.norm(vector)
    magnitude_vertical = np.linalg.norm(vertical_reference)

    # Calculate the cosine of the angle
    cos_angle = dot_product / (magnitude_vector * magnitude_vertical)

    # Clamp the value to the range [-1, 1] to avoid numerical issues with arccos
    cos_angle = np.clip(cos_angle, -1.0, 1.0)

    # Calculate the angle in degrees
    angle = np.arccos(cos_angle) * (180.0 / np.pi)
    

    return angle

In [7]:
# Calculating the state of the person

cap = cv2.VideoCapture(0)

# Curl counter variables
counter = 0
stage = None
state = "Standing"

# Setup Mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get coordinates
            hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                   landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,
                    landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,
                     landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
            
            # Calculate angle
            angle = calculate_angle(hip, knee, ankle)
            
            # Visualize angle
            cv2.putText(image, str(int(angle)), 
                        tuple(np.multiply(knee, [640, 480]).astype(int)), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            
            # State logic
            if angle < 160:
                state = "Going Down"
            elif angle > 160:
                state = "Standing"
                       
        except:
            pass
        
        # Setup status box
        cv2.rectangle(image, (0, 0), (225, 73), (245, 117, 16), -1)
        
        # Rep data
        cv2.putText(image, 'REPS', (15, 12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
        cv2.putText(image, str(counter), 
                    (10, 60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Stage data
        cv2.putText(image, 'STATE', (65, 12), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
        cv2.putText(image, state, 
                    (60, 60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2), 
                                  mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2))               
        
        cv2.imshow('Mediapipe Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [25]:
# rendering Condition 1

cap = cv2.VideoCapture(0)

# Curl counter variables
counter = 0
stage = "Good Posture"
state = "Standing"

# Setup Mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get coordinates
            hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                   landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,
                    landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,
                     landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
            shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                   landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            
            
            # Calculate angle
            angle = calculate_angle(hip, knee, ankle)
            shv_angle=calculate_angle_with_vertical(shoulder, hip)
            
            # Visualize angle
            cv2.putText(image, str(int(shv_angle)), 
                        tuple(np.multiply(knee, [640, 480]).astype(int)), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            
            # condition 1
            if angle < 160:
                state = "Going Down"
                if angle<=90:            #counting the squats
                    counter =counter+1
                if shv_angle<160:
                    stage = "Bend Backwards"
                elif shv_angle>170:
                    stage= "Bend Forward"
                else:
                    stage = "Good Posture"
                
            elif angle > 160:
                state = "Standing"
                stage = "Good Posture"
                       
        except:
            pass
        
        # Setup status box
        cv2.rectangle(image, (0, 0), (1800, 100), (245, 117, 16), -1)
        
        # Rep data
        # cv2.putText(image, 'REPS', (15, 20), 
        #             cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
        #Stage -> Condition 0
        cv2.putText(image, stage, 
                    (15, 60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)
        
        # State data
        # cv2.putText(image, 'STATE', (150, 20), 
        #             cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
        cv2.putText(image, state, 
                    (700, 60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)

        cv2.putText(image, str(counter), 
                    (1200, 60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2), 
                                  mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2))
        
        cv2.imshow('Mediapipe Feed', image)
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

In [5]:
# Adding the visibility Param
cap = cv2.VideoCapture(0)

# Curl counter variables
counter = 0
tok= None   
stage = "Good Posture"
state = "Standing"
message = ""
tok= 0   #for counter

# Setup Mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Extract landmarks
        try:
            landmarks = results.pose_landmarks.landmark
            
            # Get coordinates for left side
            hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                   landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
            knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,
                    landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
            ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,
                     landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
            shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                        landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            
            # Check if all points are visible
            if all([landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].visibility > 0.5,
                    landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].visibility > 0.5,
                    landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].visibility > 0.5,
                    landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].visibility > 0.5]):
                
                # Calculate angles
                angle = calculate_angle(hip, knee, ankle)
                shv_angle = calculate_angle_with_vertical(shoulder, hip)
                
                # Visualize shv_angle
                cv2.putText(image, str(int(shv_angle)), 
                            tuple(np.multiply(knee, [640, 480]).astype(int)), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
                
                # Condition 1
                if angle < 160:
                    state = "Going Down"
                    # tok=0
                    if angle <= 90:            # Counting the squats
                        # counter += 1
                        tok=1  
                    if shv_angle < 150:
                        stage = "Bend Backwards"
                    elif shv_angle > 165:
                        stage = "Bend Forward"
                    else:
                        stage = "Good Posture"
                
                elif angle > 160 :
                    if tok==1:   #counter function
                        counter+=1
                        tok=0
                    state = "Standing"
                    stage = "Good Posture"
                    # count=0
                
                message = ""  # Clear message if all landmarks are visible

            else:
                message = "Body not fully visible"
                stage = ""
                state = ""

        except:
            pass

        

        
        # Setup status box
        cv2.rectangle(image, (0, 0), (1800, 100), (245, 117, 16), -1)
        
        # Display message if any
        cv2.putText(image, message, 
                    (10, 500), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Rep data
        cv2.putText(image, "Reps: " + str(counter), 
                    (15, 60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Stage data
        cv2.putText(image, stage, 
                    (500, 60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)
        
        # State data
        cv2.putText(image,  state, 
                    (1000, 60), 
                    cv2.FONT_HERSHEY_SIMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Render detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2), 
                                  mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2))
        
        cv2.imshow('Mediapipe Feed', image)
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

print(counter)

8
